In [1]:
%%classpath add mvn
org.nd4j nd4j-native-platform 0.7.2
org.nd4j nd4s_2.11 0.7.2

In [2]:
%classpath add jar ../target/scala-2.11/scala-miniflow_2.11-0.1.0-SNAPSHOT.jar

In [4]:
import org.nd4j.linalg.factory.Nd4j
import org.nd4s.Implicits._
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.ops.transforms.Transforms.{sigmoid,exp,log,pow,sqrt}
import org.nd4s.Implicits._

import com.github.timsetsfire.nn.node._
import com.github.timsetsfire.nn.activation._
import com.github.timsetsfire.nn.costfunctions._
import com.github.timsetsfire.nn.regularization._
import com.github.timsetsfire.nn.optimize._
import com.github.timsetsfire.nn.graph._

import org.nd4j.linalg.factory.Nd4j
import org.nd4s.Implicits._
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.ops.transforms.Transforms.{sigmoid, exp, log, pow, sqrt}
import org.nd4s.Implicits._
import com.github.timsetsfire.nn.node._
import com.github.timsetsfire.nn.activation._
import com.github.timsetsfire.nn.costfunctions._
import com.github.timsetsfire.nn.regularization._
import com.github.timsetsfire.nn.optimize._
import com.github.timsetsfire.nn.graph._


## Create the Network

In [5]:
val x = new Input()
val y = new Input()
val h1 = ReLU(x, (13,6))
val yhat = Linear(h1, (6,1))
val mse = new MSE(y, yhat)
val network = topologicalSort{
  buildGraph(mse)
}

[[Input@6fac4881, Variable@7a56e685, Variable@3d17dbd9, Variable@22b9e63, Input@7e9c834b, Variable@77b350b2, Linear@75b00ba7, ReLU@77c91949, Linear@4b53275a, MSE@3b6c993]]

## Initialize the trainables

In [20]:
network.filter{
    _.getClass.getSimpleName == "Variable"
}.foreach{
    weights =>
      val size = weights.size
      val (m,n) = (size._1.asInstanceOf[Int], size._2.asInstanceOf[Int])
      weights.value = Nd4j.randn(m,n) * math.sqrt(3/(m.toDouble + n.toDouble))
}

null

## Import and prepare data

In [13]:
val x_ = Nd4j.readNumpy("../resources/boston_x.csv", ",")
val y_ = Nd4j.readNumpy("../resources/boston_y.csv", ",")

val Array(nobs, nfeatures) = x_.shape

// standardize data
val xs_ = x_.subRowVector(x_.mean(0)).divRowVector( x_.std(0))
val ys_ = y_.subRowVector(y_.mean(0)).divRowVector( y_.std(0))

// concatenate data
val data = Nd4j.concat(1, ys_, xs_);

OutputCell.HIDDEN

## Set hyperparameters

In [25]:
val epochs = 500
val batchSize = 100
val stepsPerEpoch = nobs / batchSize

OutputCell.HIDDEN

## Initialize Optmiizer

In [22]:
val sgd = new GradientDescent(network, learningRate = 0.1)

com.github.timsetsfire.nn.optimize.GradientDescent@61c9d3e9

In [26]:
def r2(y: INDArray, yhat: INDArray) = {
  val rss = pow(y sub yhat,2)
  val tss = pow(y.subRowVector(y.mean(0)),2)
  1d - rss.sumT / tss.sumT
}

r2: (y: org.nd4j.linalg.api.ndarray.INDArray, yhat: org.nd4j.linalg.api.ndarray.INDArray)Double


## Train network via SGD

In [24]:
for(epoch <- 0 to epochs) {
  var loss = 0d
  for(j <- 0 until stepsPerEpoch) {

    Nd4j.shuffle(data, 1)

    val feedDict: Map[Node, Any] = Map(
      x -> data.getColumns( (1 to nfeatures):_*).getRows((0 until batchSize):_*),
      y -> data.getColumn(0).getRows((0 until batchSize):_*)
    )

    sgd.optimize(feedDict)

    loss += mse.value(0,0)
  }
  if(epoch % 50 == 0)  println(s"Epoch: ${epoch}, Loss: ${loss/stepsPerEpoch.toDouble}, R^2: ${r2(y.value, yhat.value)}")
}
OutputCell.HIDDEN

Epoch: 0, Loss: 1.9552069783210755, R^2: 0.09255545763490536
Epoch: 50, Loss: 0.11243275701999664, R^2: 0.8955818783651585
Epoch: 100, Loss: 0.16669582724571227, R^2: 0.8370410591782594
Epoch: 150, Loss: 0.11385386288166047, R^2: 0.864772825942544
Epoch: 200, Loss: 0.12097328156232834, R^2: 0.8620823103992047
Epoch: 250, Loss: 0.10993571430444718, R^2: 0.880605883263533
Epoch: 300, Loss: 0.10807902812957763, R^2: 0.9454215783958917
Epoch: 350, Loss: 0.10221610963344574, R^2: 0.9045215060716915
Epoch: 400, Loss: 0.09244593977928162, R^2: 0.8720988368258396
Epoch: 450, Loss: 0.0783352643251419, R^2: 0.8749555553949862
Epoch: 500, Loss: 0.09016990810632705, R^2: 0.9058000814436297


## Calc total loss and accuracy

In [32]:
x.forward(xs_)
y.forward(ys_)
network.foreach(_.forward())
println(f"Loss: ${mse.value}, R^2: ${r2(y.value, yhat.value)}%2.2f")
OutputCell.HIDDEN

Loss: 0.09, R^2: 0.91
